<a href="https://colab.research.google.com/github/JH-KIM-82/Final-Team1/blob/main/API_%EC%9E%A5%EB%A5%B4_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 장르 불러오기

In [ ]:
# ==============================
# 1. 기본 세팅
# ==============================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 데이터 불러오기 (파일명: weighted_score_above_08_modified.csv)
df = pd.read_csv("/content/weighted_score_above_08_modified.csv")

/tmp/ipython-input-3985894152.py:2: DtypeWarning: Columns (24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/weighted_score_above_08_modified.csv")


In [ ]:
# # 속도를 빠르게 하기 위하여 유니크 앱 아이디와 게임명만 따로 추출
# # Extract unique combinations of 'appid' and 'game'
# unique_appid_game = df[['appid', 'game']].drop_duplicates()

# # Define the output file name
# output_filename = "unique_appid_game.csv"

# # Save the unique combinations to a CSV file
# unique_appid_game.to_csv(output_filename, index=False)

# print(f"Unique appid and game combinations saved to {output_filename}")

In [ ]:
import pandas as pd
import os

# 고유 appid만 추출
unique_appid = df[['appid']].drop_duplicates().reset_index(drop=True)

# 배치 크기와 기본 파일명 설정
batch_size = 5000
base_filename = "unique_appid_part"

# 출력 폴더(옵션): 필요 시 생성
output_dir = "."
os.makedirs(output_dir, exist_ok=True)

# 전체 개수
n = len(unique_appid)

# 배치 저장 루프
for i in range(0, n, batch_size):
    batch = unique_appid.iloc[i:i+batch_size].copy()
    part_idx = i // batch_size + 1  # 1부터 시작하는 파트 번호

    # 파일명에 파트 번호와 범위를 명시(가독성용)
    start_idx = i
    end_idx = min(i + batch_size, n) - 1
    filename = f"{base_filename}_{part_idx:03d}_{start_idx:06d}-{end_idx:06d}.csv"

    # CSV 저장
    batch.to_csv(os.path.join(output_dir, filename), index=False)

    print(f"Saved {len(batch)} rows to {filename}")

print(f"Done. Total rows: {n}, files created: {(n + batch_size - 1) // batch_size}")

Saved 5000 rows to unique_appid_part_001_000000-004999.csv
Saved 5000 rows to unique_appid_part_002_005000-009999.csv
Saved 5000 rows to unique_appid_part_003_010000-014999.csv
Saved 3439 rows to unique_appid_part_004_015000-018438.csv
Done. Total rows: 18439, files created: 4


In [ ]:
# 속도를 빠르게 하기 위하여 유니크 앱 아이디만 따로 추출
# Extract unique combinations of 'appid'
unique_appid_game = df[['appid']].drop_duplicates()

# Define the output file name
output_filename = "unique_appid.csv"

# Save the unique combinations to a CSV file
unique_appid_game.to_csv(output_filename, index=False)

print(f"Unique appid and game combinations saved to {output_filename}")

In [ ]:
# 호출 테스트 - 1개  불러오기
import requests

appid = 1015710
url = f"https://store.steampowered.com/api/appdetails?appids={appid}"

res = requests.get(url, timeout=10)
data = res.json()

if data.get(str(appid), {}).get("success"):
    genres = data[str(appid)]["data"].get("genres", [])
    genres = ";".join([g["description"] for g in genres])
else:
    genres = ""

print(f"{appid} 장르: {genres}")


In [ ]:
# import requests
# import csv
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import time

# INPUT_FILE = "unique_appid.csv"   # appid 리스트 (1열에 appid만 있음)
# OUTPUT_FILE = "appid_genres.csv"

# def get_genres(appid):
#     url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
#     try:
#         res = requests.get(url, timeout=5)
#         data = res.json()
#         if data.get(str(appid), {}).get("success"):
#             genres = data[str(appid)]["data"].get("genres", [])
#             return ";".join([g["description"] for g in genres])
#     except Exception as e:
#         return ""
#     return ""

# # --- 입력 데이터 읽기 ---
# with open(INPUT_FILE, "r", encoding="utf-8") as fin:
#     appids = [line.strip() for line in fin if line.strip().isdigit()]

# # --- 출력 CSV 준비 ---
# with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as fout:
#     writer = csv.writer(fout)
#     writer.writerow(["appid", "genres"])

#     # 병렬 처리
#     with ThreadPoolExecutor(max_workers=15) as executor:
#         future_to_appid = {executor.submit(get_genres, appid): appid for appid in appids}

#         for i, future in enumerate(as_completed(future_to_appid), 1):
#             appid = future_to_appid[future]
#             genres = future.result()
#             writer.writerow([appid, genres])

#             if i % 500 == 0:
#                 print(f"{i} / {len(appids)} 완료")


In [ ]:
# 장르 불러오기 - 파일로 저장 / 실패 리스트 저장
import requests
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

INPUT_FILE = "unique_appid_part_001_000000-004999.csv"   # appid 리스트 (1열에 appid만 있음)
OUTPUT_FILE = "appid_genres_1.csv"
FAILED_FILE = "failed_appid_1.csv"

# --- API 호출 함수 ---
def get_genres(appid, retries=3, delay=2):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    for attempt in range(retries):
        try:
            res = requests.get(url, timeout=10)
            data = res.json()
            if data.get(str(appid), {}).get("success"):
                genres = data[str(appid)]["data"].get("genres", [])
                return ";".join([g["description"] for g in genres])
        except Exception as e:
            # 네트워크/파싱 오류 → 재시도
            time.sleep(delay * (attempt + 1))
    return ""  # 끝까지 실패하면 빈값 리턴


# --- 입력 데이터 읽기 ---
with open(INPUT_FILE, "r", encoding="utf-8") as fin:
    appids = [line.strip() for line in fin if line.strip().isdigit()]

# --- 출력 CSV 준비 ---
with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as fout, \
     open(FAILED_FILE, "w", newline="", encoding="utf-8") as ffail:

    writer = csv.writer(fout)
    fail_writer = csv.writer(ffail)

    writer.writerow(["appid", "genres"])
    fail_writer.writerow(["appid"])  # 실패 appid 기록용

    # 병렬 처리
    with ThreadPoolExecutor(max_workers=15) as executor:
        future_to_appid = {executor.submit(get_genres, appid): appid for appid in appids}

        for i, future in enumerate(as_completed(future_to_appid), 1):
            appid = future_to_appid[future]
            genres = future.result()

            if genres:
                writer.writerow([appid, genres])
            else:
                fail_writer.writerow([appid])

            # 버퍼를 즉시 파일에 쓰기
            fout.flush()
            ffail.flush()

            if i % 500 == 0:
                print(f"{i} / {len(appids)} 완료")


500 / 5000 완료
1000 / 5000 완료
1500 / 5000 완료
2000 / 5000 완료
2500 / 5000 완료
3000 / 5000 완료
3500 / 5000 완료
4000 / 5000 완료
4500 / 5000 완료
5000 / 5000 완료


In [ ]:
# 실패한 리스트만 다시 불러오기 - 1185
import requests
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

FAILED_FILE = "failed_appid_1.csv"
OUTPUT_FILE = "appid_genres_1.csv"
RETRY_FAILED_FILE = "failed_appid_retry_1.csv"

def get_genres(appid, retries=3, delay=2):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    for attempt in range(retries):
        try:
            res = requests.get(url, timeout=10)
            data = res.json()
            if data.get(str(appid), {}).get("success"):
                genres = data[str(appid)]["data"].get("genres", [])
                return ";".join([g["description"] for g in genres])
        except Exception:
            time.sleep(delay * (attempt + 1))
    return ""


# --- 실패 appid 읽기 ---
with open(FAILED_FILE, "r", encoding="utf-8") as fin:
    next(fin)  # 헤더 건너뛰기
    appids = [line.strip() for line in fin if line.strip().isdigit()]

print(f"재시도 대상 appid 개수: {len(appids)}")

# --- 출력 파일 append 모드로 열기 ---
with open(OUTPUT_FILE, "a", newline="", encoding="utf-8") as fout, \
     open(RETRY_FAILED_FILE, "w", newline="", encoding="utf-8") as ffail:

    writer = csv.writer(fout)
    fail_writer = csv.writer(ffail)

    fail_writer.writerow(["appid"])  # 재시도 후에도 실패한 appid 기록

    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_appid = {executor.submit(get_genres, appid): appid for appid in appids}

        for i, future in enumerate(as_completed(future_to_appid), 1):
            appid = future_to_appid[future]
            genres = future.result()

            if genres:
                writer.writerow([appid, genres])
            else:
                fail_writer.writerow([appid])

            fout.flush()
            ffail.flush()

            if i % 200 == 0:
                print(f"{i} / {len(appids)} 재시도 완료")


재시도 대상 appid 개수: 3815
200 / 3815 재시도 완료
400 / 3815 재시도 완료
600 / 3815 재시도 완료
800 / 3815 재시도 완료
1000 / 3815 재시도 완료
1200 / 3815 재시도 완료
1400 / 3815 재시도 완료
1600 / 3815 재시도 완료
1800 / 3815 재시도 완료
2000 / 3815 재시도 완료
2200 / 3815 재시도 완료
2400 / 3815 재시도 완료
2600 / 3815 재시도 완료
2800 / 3815 재시도 완료
3000 / 3815 재시도 완료
3200 / 3815 재시도 완료
3400 / 3815 재시도 완료
3600 / 3815 재시도 완료
3800 / 3815 재시도 완료


In [ ]:
# 장르 불러오기 - 파일로 저장 / 실패 리스트 저장
import requests
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

INPUT_FILE = "unique_appid_part_002_005000-009999.csv"   # appid 리스트 (1열에 appid만 있음)
OUTPUT_FILE = "appid_genres_2.csv"
FAILED_FILE = "failed_appid_2.csv"

# --- API 호출 함수 ---
def get_genres(appid, retries=3, delay=2):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    for attempt in range(retries):
        try:
            res = requests.get(url, timeout=10)
            data = res.json()
            if data.get(str(appid), {}).get("success"):
                genres = data[str(appid)]["data"].get("genres", [])
                return ";".join([g["description"] for g in genres])
        except Exception as e:
            # 네트워크/파싱 오류 → 재시도
            time.sleep(delay * (attempt + 1))
    return ""  # 끝까지 실패하면 빈값 리턴


# --- 입력 데이터 읽기 ---
with open(INPUT_FILE, "r", encoding="utf-8") as fin:
    appids = [line.strip() for line in fin if line.strip().isdigit()]

# --- 출력 CSV 준비 ---
with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as fout, \
     open(FAILED_FILE, "w", newline="", encoding="utf-8") as ffail:

    writer = csv.writer(fout)
    fail_writer = csv.writer(ffail)

    writer.writerow(["appid", "genres"])
    fail_writer.writerow(["appid"])  # 실패 appid 기록용

    # 병렬 처리
    with ThreadPoolExecutor(max_workers=15) as executor:
        future_to_appid = {executor.submit(get_genres, appid): appid for appid in appids}

        for i, future in enumerate(as_completed(future_to_appid), 1):
            appid = future_to_appid[future]
            genres = future.result()

            if genres:
                writer.writerow([appid, genres])
            else:
                fail_writer.writerow([appid])

            # 버퍼를 즉시 파일에 쓰기
            fout.flush()
            ffail.flush()

            if i % 500 == 0:
                print(f"{i} / {len(appids)} 완료")


500 / 5000 완료
1000 / 5000 완료
1500 / 5000 완료
2000 / 5000 완료
2500 / 5000 완료
3000 / 5000 완료
3500 / 5000 완료
4000 / 5000 완료
4500 / 5000 완료
5000 / 5000 완료


In [ ]:
# 실패한 리스트만 다시 불러오기
import requests
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

FAILED_FILE = "failed_appid_1.csv"
OUTPUT_FILE = "appid_genres_1.csv"
RETRY_FAILED_FILE = "failed_appid_retry_1.csv"

def get_genres(appid, retries=3, delay=2):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    for attempt in range(retries):
        try:
            res = requests.get(url, timeout=10)
            data = res.json()
            if data.get(str(appid), {}).get("success"):
                genres = data[str(appid)]["data"].get("genres", [])
                return ";".join([g["description"] for g in genres])
        except Exception:
            time.sleep(delay * (attempt + 1))
    return ""


# --- 실패 appid 읽기 ---
with open(FAILED_FILE, "r", encoding="utf-8") as fin:
    next(fin)  # 헤더 건너뛰기
    appids = [line.strip() for line in fin if line.strip().isdigit()]

print(f"재시도 대상 appid 개수: {len(appids)}")

# --- 출력 파일 append 모드로 열기 ---
with open(OUTPUT_FILE, "a", newline="", encoding="utf-8") as fout, \
     open(RETRY_FAILED_FILE, "w", newline="", encoding="utf-8") as ffail:

    writer = csv.writer(fout)
    fail_writer = csv.writer(ffail)

    fail_writer.writerow(["appid"])  # 재시도 후에도 실패한 appid 기록

    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_appid = {executor.submit(get_genres, appid): appid for appid in appids}

        for i, future in enumerate(as_completed(future_to_appid), 1):
            appid = future_to_appid[future]
            genres = future.result()

            if genres:
                writer.writerow([appid, genres])
            else:
                fail_writer.writerow([appid])

            fout.flush()
            ffail.flush()

            if i % 200 == 0:
                print(f"{i} / {len(appids)} 재시도 완료")
